# BengaliAI CV 19 - Model & Train

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import os

In [2]:
np.random.seed(1) # to make the results reproducible

In [3]:
print(tf.test.is_gpu_available())
print(tf.__version__)
print(tf.keras.__version__)

True
2.0.0
2.2.4-tf


In [4]:
if tf.__version__.startswith("2.0"):
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        print("Configuring GPU device: {}".format(str(gpu)))
        tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_virtual_device_configuration(
            gpu,
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7*1024)]
        )
else:
    tf.config.gpu.set_per_process_memory_fraction(0.75)
    tf.config.gpu.set_per_process_memory_growth(True)

Configuring GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Model

In [5]:
# Detect the number of train data mini batches. Shuffle them and divide into a train/test parts.

fname_pattern = "./bengaliai-cv19/train_batch_{:03d}.pickle"
train_test_split = 0.7

mbatch_no = 0
while os.path.exists(fname_pattern.format(mbatch_no)):
    mbatch_no = mbatch_no + 1
print("{} train data file(s) found.".format(mbatch_no))

mbatch_train_no = round(mbatch_no * 0.7)
mbatch_test_no = mbatch_no - mbatch_train_no
mbatch_idx = np.random.permutation(mbatch_no)
mbatch_train_idx = mbatch_idx[:mbatch_train_no]
mbatch_test_idx = mbatch_idx[mbatch_train_no:]

197 train data file(s) found.


### Convolutional-Dense Autoencoder

In [6]:
# Keras model
X = tf.keras.layers.Input(shape=[137,236])
A0 = tf.keras.layers.Reshape(target_shape=[137,236,1])(X)
O0 = A0

I1 = O0
Z1 = tf.keras.layers.Conv2D(filters=256, kernel_size=[5,5], strides=[1,1], padding="valid", dilation_rate=[1,1])(I1)
A1 = tf.keras.layers.LeakyReLU(alpha=0.1)(Z1)
P1 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=[2,2], padding="same")(A1)
O1 = P1

I2 = O1
Z2 = tf.keras.layers.Conv2D(filters=32, kernel_size=[3,3], strides=[1,1], padding="valid", dilation_rate=[1,1])(I2)
A2 = tf.keras.layers.LeakyReLU(alpha=0.1)(Z2)
P2 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=[2,2], padding="same")(A2)
O2 = P2

I3 = O2
Z3 = tf.keras.layers.Conv2D(filters=32, kernel_size=[3,3], strides=[1,1], padding="valid", dilation_rate=[1,1])(I3)
A3 = tf.keras.layers.LeakyReLU(alpha=0.1)(Z3)
P3 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=[2,2], padding="same")(A3)
O3 = P3

I4 = O3
Z4 = tf.keras.layers.Conv2D(filters=32, kernel_size=[3,3], strides=[1,1], padding="valid", dilation_rate=[1,1])(I4)
A4 = tf.keras.layers.LeakyReLU(alpha=0.1)(Z4)
P4 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=[2,2], padding="same")(A4)
F4 = tf.keras.layers.Flatten()(P4)
O4 = F4

# <-- encoded here

I5 = O4
Z5 = tf.keras.layers.Dense(units=500, use_bias=True)(I5)
A5 = tf.keras.layers.LeakyReLU(alpha=0.1)(Z5)
O5 = A5

I6 = O5
Z6 = tf.keras.layers.Dense(units=2000, use_bias=True)(I6)
A6 = tf.keras.layers.LeakyReLU(alpha=0.1)(Z6)
O6 = A6

I7 = O6
Z7 = tf.keras.layers.Dense(units=236*137, use_bias=True)(I7)
R7 = tf.keras.layers.Reshape(target_shape=[137,236])(Z7)
O7 = R7

Y = O7

if tf.__version__.startswith("2.0"):
    model = tf.keras.models.Model(X, Y)
else:
    model = tf.keras.models.Model(inputs=X, outputs=Y)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 137, 236)]        0         
_________________________________________________________________
reshape (Reshape)            (None, 137, 236, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 133, 232, 256)     6656      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 133, 232, 256)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 67, 116, 256)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 65, 114, 32)       73760     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 65, 114, 32)       0     